<a href="https://colab.research.google.com/github/Hanbin-git/kaggle/blob/main/test2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install timm
!pip install scikit-learn


In [2]:
!pip install efficientnet_pytorch

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16426 sha256=21e0bb10927f21f0dfde3c3df9a318fba6d80e573018bc59d73fa11bbbbfbd26
  Stored in directory: /root/.cache/pip/wheels/8b/6f/9b/231a832f811ab6ebb1b32455b177ffc6b8b1cd8de19de70c09
Successfully built efficientnet_pytorch


In [3]:

# 2. 런타임 확인용 (설치 잘 되었는지)
import torch
import timm
print("✅ GPU 사용 가능:", torch.cuda.is_available())
print("✅ GPU 이름:", torch.cuda.get_device_name(0))

✅ GPU 사용 가능: True
✅ GPU 이름: NVIDIA A100-SXM4-40GB


In [3]:
# 전략 1. 데이터 전처리
#       - 이미지 사이즈 및 중심으로 정렬
#       - 밝기/대비 조정
#       - 이미지 노이즈 제거
# 전략 2. 데이터 증강
#       - 기본 Augmentation 차량의 각도나 배경 고려
#       - Mixup/cutmix (클래스 간 경계를 부드럽게 학습)

In [4]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import os
os.makedirs('models', exist_ok=True)
os.makedirs('val_logs', exist_ok=True)


In [6]:
import os
import random
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from efficientnet_pytorch import EfficientNet
from torch.cuda.amp import autocast, GradScaler

# ========================
# 설정
SEED = 42
BATCH_SIZE = 32
EPOCHS = 20
FOLDS = 5
IMG_SIZE = 456
PATIENCE = 3
NUM_WORKERS = 2
TTA_ROUNDS = 2
NUM_CLASSES = 396

ROOT_DIR = '/content/drive/MyDrive/open'
TRAIN_DIR = os.path.join(ROOT_DIR, 'train')
TEST_CSV = os.path.join(ROOT_DIR, 'test.csv')
SUBMIT_CSV = os.path.join(ROOT_DIR, 'sample_submission.csv')

os.makedirs("models", exist_ok=True)
os.makedirs("val_logs", exist_ok=True)

random.seed(SEED)
torch.manual_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ========================
# 클래스 통합
merge_map = {
    'K5_3세대_하이브리드_2020_2022': 'K5_하이브리드_3세대_2020_2023',
    '디_올뉴니로_2022_2025': '디_올_뉴_니로_2022_2025',
    '718_박스터_2017_2024': '박스터_718_2017_2024',
}
raw_classes = os.listdir(TRAIN_DIR)
merged_classes = sorted(set([merge_map.get(c, c) for c in raw_classes]))
class_to_idx = {c: i for i, c in enumerate(merged_classes)}
idx_to_class = {i: c for c, i in class_to_idx.items()}

# ========================
# Dataset 정의
class CarDataset(Dataset):
    def __init__(self, image_paths, labels=None, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = Image.open(self.image_paths[idx]).convert('RGB')
        if self.transform:
            img = self.transform(img)
        if self.labels is not None:
            return img, self.labels[idx]
        else:
            return img

# ========================
# Transform 정의
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# ========================
# 학습 데이터 로딩
train_image_paths, train_labels = [], []
for folder in raw_classes:
    unified = merge_map.get(folder, folder)
    for f in os.listdir(os.path.join(TRAIN_DIR, folder)):
        train_image_paths.append(os.path.join(TRAIN_DIR, folder, f))
        train_labels.append(class_to_idx[unified])

# ========================
# KFold 학습
skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=SEED)
val_acc_log = []

for fold, (train_idx, val_idx) in enumerate(skf.split(train_image_paths, train_labels)):
    print(f"\n🌀 Fold {fold+1}/{FOLDS}")
    X_train = [train_image_paths[i] for i in train_idx]
    y_train = [train_labels[i] for i in train_idx]
    X_val = [train_image_paths[i] for i in val_idx]
    y_val = [train_labels[i] for i in val_idx]

    train_ds = CarDataset(X_train, y_train, transform=train_transform)
    val_ds = CarDataset(X_val, y_val, transform=test_transform)
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

    model = EfficientNet.from_pretrained('efficientnet-b5', num_classes=NUM_CLASSES).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
    criterion = nn.CrossEntropyLoss()
    scaler = GradScaler()

    best_acc, patience_counter = 0, 0
    for epoch in range(EPOCHS):
        model.train()
        for images, labels in tqdm(train_loader, desc=f"[Fold {fold+1}][Epoch {epoch+1}]"):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            with autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

        model.eval()
        preds, true = [], []
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
                true.extend(labels.cpu().numpy())

        acc = accuracy_score(true, preds)
        print(f"📌 [Fold {fold+1}][Epoch {epoch+1}] Val Accuracy: {acc:.4f}")

        if acc > best_acc:
            best_acc = acc
            torch.save(model.state_dict(), f"models/best_fold{fold+1}.pth")
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= PATIENCE:
                print("⏹️ Early stopping triggered.")
                break

    val_acc_log.append(best_acc)

    cm = confusion_matrix(true, preds)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, cmap="Blues", xticklabels=False, yticklabels=False)
    plt.title(f"Confusion Matrix Fold {fold+1}")
    plt.savefig(f"val_logs/confmat_fold{fold+1}.png")
    plt.close()

print("✅ 학습 완료")
print(f"📊 Fold별 Val Accuracy: {val_acc_log}")
print(f"📈 평균 Validation Accuracy: {np.mean(val_acc_log):.4f}")


🌀 Fold 1/5


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b5-b6417697.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b5-b6417697.pth
100%|██████████| 117M/117M [00:04<00:00, 28.1MB/s]
<ipython-input-6-68c520df951b>:120: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


Loaded pretrained weights for efficientnet-b5


[Fold 1][Epoch 1]:   0%|          | 0/829 [00:00<?, ?it/s]<ipython-input-6-68c520df951b>:128: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
[Fold 1][Epoch 1]: 100%|██████████| 829/829 [04:56<00:00,  2.80it/s]


📌 [Fold 1][Epoch 1] Val Accuracy: 0.5054


...

📌 [Fold 5][Epoch 18] Val Accuracy: 0.9543


[Fold 5][Epoch 19]:   0%|          | 0/829 [00:00<?, ?it/s]<ipython-input-6-68c520df951b>:128: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
[Fold 5][Epoch 19]: 100%|██████████| 829/829 [04:55<00:00,  2.80it/s]


📌 [Fold 5][Epoch 19] Val Accuracy: 0.9570


[Fold 5][Epoch 20]:   0%|          | 0/829 [00:00<?, ?it/s]<ipython-input-6-68c520df951b>:128: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
[Fold 5][Epoch 20]: 100%|██████████| 829/829 [04:55<00:00,  2.81it/s]


📌 [Fold 5][Epoch 20] Val Accuracy: 0.9574
✅ 학습 완료
📊 Fold별 Val Accuracy: [0.9615268557634279, 0.959867229933615, 0.9533796016898008, 0.9607665610381771, 0.9574468085106383]
📈 평균 Validation Accuracy: 0.9586


In [3]:
import os, random
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import timm
from torch.cuda.amp import autocast, GradScaler

# ========================
# 설정
SEED = 42
BATCH_SIZE = 32
EPOCHS = 5
FOLDS = 5
TTA_ROUNDS = 2
IMG_SIZE = 224
PATIENCE = 3

os.makedirs("models", exist_ok=True)
os.makedirs("val_logs", exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(SEED)

# ========================
# 클래스 통합
merge_map = {
    'K5_3세대_하이브리드_2020_2022': 'K5_하이브리드_3세대_2020_2023',
    '디_올뉴니로_2022_2025': '디_올_뉴_니로_2022_2025',
    '718_박스터_2017_2024': '박스터_718_2017_2024',
}

ROOT_DIR = '/content/drive/MyDrive/open'
TRAIN_DIR = os.path.join(ROOT_DIR, 'train')
TEST_DIR = os.path.join(ROOT_DIR, 'test')
TEST_CSV = os.path.join(ROOT_DIR, 'test.csv')
SUBMIT_CSV = os.path.join(ROOT_DIR, 'sample_submission.csv')

raw_classes = os.listdir(TRAIN_DIR)
merged_classes = sorted(set([merge_map.get(c, c) for c in raw_classes]))
class_to_idx = {c: i for i, c in enumerate(merged_classes)}
idx_to_class = {i: c for c, i in class_to_idx.items()}

# ========================
# Dataset 정의
class CarDataset(Dataset):
    def __init__(self, image_paths, labels=None, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = Image.open(self.image_paths[idx]).convert('RGB')
        if self.transform:
            img = self.transform(img)
        if self.labels is not None:
            return img, self.labels[idx]
        else:
            return img

# ========================
# Transform
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.05),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# ========================
# 데이터 로딩
train_image_paths, train_labels = [], []
for folder in raw_classes:
    unified = merge_map.get(folder, folder)
    for f in os.listdir(os.path.join(TRAIN_DIR, folder)):
        train_image_paths.append(os.path.join(TRAIN_DIR, folder, f))
        train_labels.append(class_to_idx[unified])

test_df = pd.read_csv(TEST_CSV)
test_image_paths = [os.path.join(ROOT_DIR, p) for p in test_df['img_path']]
submission = pd.read_csv(SUBMIT_CSV)
preds_396 = np.zeros((len(test_df), 396))
val_acc_log = []

# ========================
# KFold 학습
skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=SEED)

for fold, (train_idx, val_idx) in enumerate(skf.split(train_image_paths, train_labels)):
    print(f"\n🌀 Fold {fold+1}/{FOLDS}")
    X_train = [train_image_paths[i] for i in train_idx]
    y_train = [train_labels[i] for i in train_idx]
    X_val = [train_image_paths[i] for i in val_idx]
    y_val = [train_labels[i] for i in val_idx]

    train_ds = CarDataset(X_train, y_train, transform=train_transform)
    val_ds = CarDataset(X_val, y_val, transform=test_transform)

    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

    model = timm.create_model('efficientnet_b5', pretrained=True, num_classes=len(merged_classes)).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
    criterion = nn.CrossEntropyLoss()
    scaler = GradScaler()

    best_acc, patience_counter = 0, 0
    for epoch in range(EPOCHS):
        model.train()
        for images, labels in tqdm(train_loader, desc=f"[Fold {fold+1}][Epoch {epoch+1}]"):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            with autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

        model.eval()
        preds, true = [], []
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
                true.extend(labels.cpu().numpy())
        acc = accuracy_score(true, preds)
        print(f"📌 [Fold {fold+1}][Epoch {epoch+1}] Val Accuracy: {acc:.4f}")

        if acc > best_acc:
            best_acc = acc
            torch.save(model.state_dict(), f"models/best_fold{fold+1}.pth")
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= PATIENCE:
                print("⏹️ Early stopping triggered.")
                break

    val_acc_log.append(best_acc)

    cm = confusion_matrix(true, preds)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, cmap="Blues", xticklabels=False, yticklabels=False)
    plt.title(f"Confusion Matrix Fold {fold+1}")
    plt.savefig(f"val_logs/confmat_fold{fold+1}.png")
    plt.close()

    model.load_state_dict(torch.load(f"models/best_fold{fold+1}.pth"))
    model.eval()
    fold_preds = np.zeros((len(test_df), len(merged_classes)))
    for _ in range(TTA_ROUNDS):
        test_ds = CarDataset(test_image_paths, transform=test_transform)
        test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)
        tta_preds = []
        with torch.no_grad():
            for batch in test_loader:
                imgs = batch.to(device)
                probs = torch.softmax(model(imgs), dim=1).cpu().numpy()
                tta_preds.append(probs)
        fold_preds += np.vstack(tta_preds) / TTA_ROUNDS

    for col in submission.columns[1:]:
        mapped_col = merge_map.get(col, col)
        if mapped_col in class_to_idx:
            preds_396[:, submission.columns.get_loc(col)-1] += fold_preds[:, class_to_idx[mapped_col]] / FOLDS

submission.iloc[:, 1:] = preds_396
submission.to_csv("submission_final.csv", index=False)
print("✅ 최종 제출 파일 저장 완료: submission_final.csv")
print(f"📊 Fold별 Val Accuracy: {val_acc_log}")
print(f"📈 평균 Validation Accuracy: {np.mean(val_acc_log):.4f}")



🌀 Fold 1/5


<ipython-input-3-4e131028c290>:125: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
[Fold 1][Epoch 1]:   0%|          | 0/829 [00:00<?, ?it/s]<ipython-input-3-4e131028c290>:133: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
[Fold 1][Epoch 1]:  46%|████▌     | 380/829 [01:32<01:49,  4.09it/s]


IsADirectoryError: Caught IsADirectoryError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "<ipython-input-3-4e131028c290>", line 65, in __getitem__
    img = Image.open(self.image_paths[idx]).convert('RGB')
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/PIL/Image.py", line 35